In [ ]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K

from sificc_lib import AI, utils, Event, Simulation, root_files, DataModel
np.set_printoptions(precision=2, linewidth=85, suppress=True)

%matplotlib inline

In [ ]:
# model name
model_name = 'model-2b-lvf'

shuffle_clusters = True

# load the training data
data = DataModel('data-mtx-enough-top-8.npz', 
                 batch_size = 128, validation_percent = .1, test_percent = .2, 
                 weight_compton=1.5, weight_non_compton=.5)

# append an extra dimention to the features since we are using convolutional layers
data.append_dim = True

# create an AI instance
ai = AI(data, model_name)

ai.weight_type = 2
ai.weight_pos_x = 2.5
ai.weight_pos_y = 1
ai.weight_pos_z = 2
ai.weight_energy = 1.5
ai.weight_e_cluster = 1
ai.weight_p_cluster = 1

# shuffle the clusters within each event
if shuffle_clusters:
    ai.data.shuffle_training_clusters()

In [ ]:
# define and create the neural network architecture
ai.create_model(conv_layers=[128, 64], classifier_layers=[32], type_layers=[8], 
                pos_layers=[64,32], energy_layers=[32, 16], base_l2=.000, limbs_l2=.000)

In [ ]:
# compile the ai
ai.compile_model(learning_rate=0.001)

# define the learning rate scheduler for the training phase
def lr_scheduler(epoch):
    if epoch < 110:
        return .001
    elif epoch < 180:
        return .0003
    elif epoch < 240:
        return .0001
    elif epoch < 270:
        return .00003
    elif epoch < 290:
        return .00001
    else:
        return .000003
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
]

In [ ]:
# #LOADING
# ai.load(model_name, optimizer=False)
# ai.compile_model()

In [ ]:
#TRAINING

In [ ]:
%%time
# start the training
ai.train(epochs=300, shuffle=True, shuffle_clusters=shuffle_clusters, verbose=0, callbacks = l_callbacks)

# evaluate the AI on the training set
ai.model.evaluate(ai.data.train_x, ai.data.train_y, verbose=1)
print()

In [ ]:
# plot the training loss
ai.plot_training_loss(smooth=False)

In [ ]:
# evaluate the AI on the test dataset
ai.evaluate()

In [ ]:
%%time
# tuning the clusters

# eliminate the components weight not intended for tuning
ai.weight_type = 2      * 0
ai.weight_pos_x = 2.5   * 0
ai.weight_pos_y = 1     * 0
ai.weight_pos_z = 2     * 0
ai.weight_energy = 1.5  * 0
ai.weight_e_cluster = 1 * 1
ai.weight_p_cluster = 1 * 1

# freeze all network components except for the parts to be tuned
for layer in ai.model.layers:
    if layer.name.find('cluster') != -1:
        layer.trainable = True
        print(f'activating {layer.name}')
    else:
        layer.trainable = False
        
# compile the ai
ai.compile_model(learning_rate=0.00003)

# define the learning rate scheduler for the training phase
def lr_scheduler(epoch):
    if epoch < 15:
        return .00003
    elif epoch < 25:
        return .00001
    else:
        return .000003
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
]

ai.train(epochs=30, shuffle=True, shuffle_clusters=shuffle_clusters, verbose=0, callbacks = l_callbacks)
ai.evaluate()

In [ ]:
%%time
# tuning the type

# eliminate the components weight not intended for tuning
ai.weight_type = 2      * 1
ai.weight_pos_x = 2.5   * 0
ai.weight_pos_y = 1     * 0
ai.weight_pos_z = 2     * 0
ai.weight_energy = 1.5  * 0
ai.weight_e_cluster = 1 * 0
ai.weight_p_cluster = 1 * 0

# freeze all network components except for the parts to be tuned
for layer in ai.model.layers:
    if layer.name.find('type') != -1:
        layer.trainable = True
        print(f'activating {layer.name}')
    else:
        layer.trainable = False
        
# compile the ai
ai.compile_model(learning_rate=0.00003)

# define the learning rate scheduler for the training phase
def lr_scheduler(epoch):
    if epoch < 15:
        return .00003
    elif epoch < 25:
        return .00001
    else:
        return .000003
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
]

ai.train(epochs=30, shuffle=True, shuffle_clusters=shuffle_clusters, verbose=0, callbacks = l_callbacks)
ai.evaluate()

In [ ]:
%%time
# tuning the posistions

# eliminate the components weight not intended for tuning
ai.weight_type = 2      * 0
ai.weight_pos_x = 2.5   * 1
ai.weight_pos_y = 1     * 1
ai.weight_pos_z = 2     * 1
ai.weight_energy = 1.5  * 0
ai.weight_e_cluster = 1 * 0
ai.weight_p_cluster = 1 * 0

# freeze all network components except for the parts to be tuned
for layer in ai.model.layers:
    if layer.name.find('pos') != -1:
        layer.trainable = True
        print(f'activating {layer.name}')
    else:
        layer.trainable = False
        
# compile the ai
ai.compile_model(learning_rate=0.00003)

# define the learning rate scheduler for the training phase
def lr_scheduler(epoch):
    if epoch < 15:
        return .00003
    elif epoch < 25:
        return .00001
    else:
        return .000003
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
]

ai.train(epochs=30, shuffle=True, shuffle_clusters=shuffle_clusters, verbose=0, callbacks = l_callbacks)
ai.evaluate()

In [ ]:
%%time
# tuning the energy

# eliminate the components weight not intended for tuning
ai.weight_type = 2      * 0
ai.weight_pos_x = 2.5   * 0
ai.weight_pos_y = 1     * 0
ai.weight_pos_z = 2     * 0
ai.weight_energy = 1.5  * 1
ai.weight_e_cluster = 1 * 0
ai.weight_p_cluster = 1 * 0

# freeze all network components except for the parts to be tuned
for layer in ai.model.layers:
    if layer.name.find('energy') != -1:
        layer.trainable = True
        print(f'activating {layer.name}')
    else:
        layer.trainable = False
        
# compile the ai
ai.compile_model(learning_rate=0.00003)

# define the learning rate scheduler for the training phase
def lr_scheduler(epoch):
    if epoch < 15:
        return .00003
    elif epoch < 25:
        return .00001
    else:
        return .000003
l_callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler),
]

ai.train(epochs=30, shuffle=True, shuffle_clusters=shuffle_clusters, verbose=0, callbacks = l_callbacks)
ai.evaluate()

In [ ]:
# save the trained model
ai.save(file_name=model_name)